In [23]:
#put unitary representations of the symmetric group over GF(q^2) together to form unitary DFT
#if each rep'n is unitary, and we use normalizing factors \sqrt{d_\rho/|G|}, the overall matrix should be unitary

In [46]:
#define conjugation as x |--> x**q, an order two automorphism of F_q^2. note x**q == x for x \in F_q.
def conjugate_pos_char(A):
    return matrix(F,[[A[i][j]**q for j in range(A.nrows())] for i in range(A.nrows())])

In [45]:
def invariant_symmetric_bilinear_matrix(q,partition):
    """
    Computes the matrix of a S_n-invariant symmetric bilinear form.
    Sets up and solves system of linear equations based on writing U as an unknown in polynomial ring generators. 
    The equations are \rho(g)^T*U*\overline{\rho(g)} = \lambda_g*U where \lambda_g = \det(\rho(g))\overline{\det(\rho(g))}.
    The variables for U can be extracted to yield a matrix over GF(q^2) for each g.
    These are stacked to get the overall system, and we find the one dim'l null space to get a solution vector, and format as a matrix.
    Note: one could also form the Kroenecker products \rho(g) \otimes \rho(g)^{-1 T} to explicitly obtain the system.
    """
    # Define the group G and its rep'n as a Specht module, dimension
    n = sum(partition)
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    d_rho = specht_module.dimension()
    
    # Initialize U as a matrix of variables over GF(q^2)
    R = PolynomialRing(F, 'u', d_rho**2)
    U_vars = R.gens()  # List of variable generators for U
    U = matrix(R, d_rho, d_rho, U_vars)  # U is a d_rho x d_rho matrix of variables
    
    # for each generator of G, form the augmented system 
    def augmented_matrix(g):
    
        #form the matrix equation \rho(g)^T*U*\overline{\rho(g)} = \lambda_g * U
        #lambda_g = rho_g.det()*rho_g.det()**q
        #note: \lambda_g isn't necessary here, and probably only relates to sesquilinear forms
        equation_matrix = rho(g).transpose()*U*conjugate_pos_char(rho(g)) - U
    
        # Initialize a list to hold rows of the augmented system
        augmented_system = []
    
        # Extract coefficients for each linear equation in the matrix
        for i in range(d_rho):
            for j in range(d_rho):
                # Get the (i, j) entry of the equation matrix, which is a linear combination of the u variables
                linear_expression = equation_matrix[i, j]
            
                # Extract the coefficients of each u_k in the linear expression
                row = [linear_expression.coefficient(u) for u in U_vars]
            
                # Append the row to the augmented system
                augmented_system.append(row)
    
        # Convert the augmented system to a matrix
        return matrix(F, augmented_system)

    #stack linear systems for each g in G
    total_system = matrix(F,0,d_rho**2)
    for g in G:
        total_system = total_system.stack(augmented_matrix(g))
    
    #compute the null space of the overall matrix
    null_space = total_system.right_kernel()
    
    #return a d_rho x d_rho matrix over GF(q^2) from the 1 dim'l null space given as vector
    U_mats = [matrix(F,d_rho,d_rho,b) for b in null_space.basis()]

    #verify that a solution to the linear system satisfies the G-invariance property
    assert all(rho(g).transpose()*U_mats[0]*conjugate_pos_char(rho(g)) == U_mats[0] for g in G)
    
    return U_mats

In [44]:
#for u in GF(q), we can factor as u=aa^* using gen. z and modular arithmetic
def factor_scalar(u):
    if u == 0:
        return 0  # Special case for 0
    z = GF(q**2).multiplicative_generator()
    k = u.log(z)  # Compute discrete log of u to the base z
    if k % (q+1) != 0:
        raise ValueError("Unable to factor: u.log(z) is not divisible by q+1, i.e. u is not in base field GF(q)")
    return z**((k//(q+1))%(q-1))

In [43]:
#use libgap.eval for GAP evalutation of BaseChangeToCanonical using `forms` package
def unitary_change_of_basis(U,q):
    if U.nrows() == 1 and U.ncols() == 1:
        return matrix(F,[[factor_scalar(U[0,0])]])
    libgap.LoadPackage("forms")
    return matrix(GF(q**2),libgap.BaseChangeToCanonical(libgap([list(row) for row in U]).HermitianFormByMatrix(GF(q^2)))).inverse()

In [52]:
#define the Fourier coefficient at the rep'n specht_module corresponding to partition
def hat(f,partition,unitary=False):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    if unitary:
        U = invariant_symmetric_bilinear_matrix(q,partition)[0]
        A = unitary_change_of_basis(U,q)
        A_star = conjugate_pos_char(A).transpose()
        sqrt_unitary_factor = sqrt(F(specht_module.dimension()/G.cardinality()))
        return sqrt_unitary_factor*sum(f(g)*A_star*rho(g)*A_star.inverse() for g in G)
    else:
        return sum(f(g)*rho(g) for g in G)

In [53]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
from sage.misc.flatten import flatten
delta = lambda s: lambda t: 1 if t == s else 0 #delta function \delta_s(t)
def dft(unitary=False):
    fourier_transform = [flatten([hat(delta(g),partition,unitary).list() for partition in Partitions(G.degree())]) for g in G]
    if unitary:
        dft_matrix = matrix(F,fourier_transform).transpose()
        sign_diag = (dft_matrix*conjugate_pos_char(dft_matrix).transpose()).diagonal()
        factor_diag = diagonal_matrix([factor_scalar(d) for d in sign_diag])
        return factor_diag.inverse()*dft_matrix
    else:
        return matrix(fourier_transform).transpose()

In [54]:
#parameters and define the symmetric group algebra
n = 4; q = 11
SGA = SymmetricGroupAlgebra(GF(q**2), n)
F = SGA.base_ring()
G = SGA.group()

In [55]:
#compute the unitary DFT
unitary_dft = dft(unitary=True); print(unitary_dft)

[     5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2      5*z2]
[     8*z2      8*z2      4*z2      4*z2      4*z2      4*z2      3*z2      3*z2      7*z2      7*z2      7*z2      7*z2      7*z2      7*z2      4*z2      4*z2         0         0      7*z2      7*z2      4*z2      4*z2         0         0]
[        0         0  6*z2 + 5  2*z2 + 9  6*z2 + 5  2*z2 + 9         0         0  5*z2 + 6  9*z2 + 2  5*z2 + 6  9*z2 + 2  6*z2 + 5  2*z2 + 9  5*z2 + 6  9*z2 + 2  7*z2 + 4  4*z2 + 7  6*z2 + 5  2*z2 + 9  5*z2 + 6  9*z2 + 2  7*z2 + 4  4*z2 + 7]
[        0         0         0 10*z2 + 7         0 10*z2 + 7         0         0         0    z2 + 4         0    z2 + 4         0 10*z2 + 7         0    z2 + 4 10*z2 + 7    z2 + 4         0 10*z2 + 7         0    z2 + 4 10*z2 + 7    z2 + 4]
[        0         0  5*z2 + 6  

In [49]:
#verify the resulting DFT is unitary
unitary_dft*conjugate_pos_char(unitary_dft).transpose() == identity_matrix(SGA.group().cardinality())

True